# Incorporating live orders into your holdings view

In this example we demonstrate how to give front office users a view of intraday trading activity on top of their middle office IBOR. We set up a portfolio with multiple strategies and generate orders and allocations. We can show updated positions that account for these live, partially allocated orders.

To illustrate, below we will trade <i>Amazon, Inc.</i>. We'll create a set of limit and market orders, and show an updated view of the portfolio intraday.


## Imports

In [1]:
import lusid
import lusid.models as models
import lusid.api as la
from lusid import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)
import uuid
from lusidtools.lpt.lpt import to_date
from collections import defaultdict
import pandas as pd
import numpy as np
import json
import openpyxl
import os
from datetime import date,timedelta,datetime

pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 500)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.6.10817.0


Define scope, portfolio and some other variables used in the example:

In [2]:
orders_scope = "holdings_with_live_orders"
orders_portfolio = "us_long"
strategy_shk = "strategy"
date_today=date.today
pf_created_date = "2020-01-01T00:00:00+00:00"

recipe_scope = orders_scope
recipe_code = "test_recipe"

## Create a Portfolio

In [3]:
def create_portfolio(scope, portfolio_code, name):

    pf_df = pd.DataFrame(data=[
        {"portfolio_code": portfolio_code, "portfolio_name": name},
    ])
    
    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "$USD",
        },
        "optional": {
            "created": f"${pf_created_date}"
        },
    }
    
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
    )

    succ, failed = format_portfolios_response(result)
    display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}])) 

In [4]:
    create_portfolio(orders_scope, orders_portfolio, "Holdings + Live")

,success,failed
0,1,0


## Load an Instrument Master with a single instrument

In [5]:
instr_df = pd.read_csv("data/live_orders_instruments.csv")
display(instr_df)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Anglian_Water_40LV,imd_13579246,USD,XS0089553282,NaN,GB,united_kingdom,NaN,fixed_income,bond,6.625
2,ICE_Silver_5000oz_Dec21,imd_12457801,USD,NaN,NaN,GB,united_kingdom,ZIZ21,commodity,future,NaN


In [6]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "client_internal",
        "Isin": "isin",
        "Figi": "figi",
    },
    "required": {
        "name": "instrument_name"
    },
}

In [7]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=orders_scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)
# display(result)
succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,3,0,0


## Create a recipe

In [8]:
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

configuration_recipe = models.ConfigurationRecipe(
    recipe_scope,
    recipe_code,
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=orders_scope,
                quote_type="Price",
                field="mid",
            )
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=orders_scope
        )
    ),
    pricing=models.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": False},
    ),
)

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

## Set initial holdings in the parent portfolio

In [9]:
hldgs_df = pd.read_csv("data/initial_amazon_holdings.csv")
display(hldgs_df)

,instrument_name,client_internal,isin,figi,quantity,unit_cost,total_cost,currency
0,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,5000,165,825000,USD


In [10]:
holdings_mapping = {
    "required":{
        "code": f"${orders_portfolio}",
        "effective_at": "$2020-05-01",
        "tax_lots.units": "quantity"
    },
    "identifier_mapping": {
        "ClientInternal": "client_internal",
    },
    "optional": {
        "tax_lots.cost.amount": "total_cost",
        "tax_lots.cost.currency": "currency",
        "tax_lots.price": "unit_cost"
    }
}

In [11]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=orders_scope,
    data_frame=hldgs_df,
    mapping_required=holdings_mapping["required"],
    mapping_optional=holdings_mapping["optional"],
    identifier_mapping=holdings_mapping["identifier_mapping"],
    file_type="holdings"
)
succ, failed = format_holdings_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


In [12]:
# Read some equity prices from EDI (DD addition)
equity_prices_df = pd.read_csv("data/edi.csv", keep_default_na = False)
display(equity_prices_df)


,Asset,client_internal,Figi,Date,Price,Currency
0,Amazon_Nasdaq_AMZN,imd_34634534,BBG000BVPXP1,2023-01-17,96.0,USD
1,Amazon_Nasdaq_AMZN,imd_34634534,BBG000BVPXP1,2023-01-23,97.0,USD


In [13]:
# Obtain the Quotes API

quotes_api = api_factory.build(la.QuotesApi)

# Create convenience function
def load_quotes(dataframe, provider, identifier, par):

    # Iterate over prices in the dataframe, creating one quote definition per row
    quotes = {
        # Each quote must be upserted with an ephemeral key (uuid in this case), to track errors in the response
        str(uuid.uuid4()): models.UpsertQuoteRequest(
            quote_id = models.QuoteId(
                quote_series_id = models.QuoteSeriesId(
                    # Must be one of the valid financial data vendor 'provider' values
                    provider = provider,
                    # An identifier type; must be one of Figi, Isin, RIC, ClientInternal, LusidInstrumentId, QuotePermId
                    instrument_id_type = identifier,
                    # A valid value for the chosen identifier type 
                    instrument_id = price[identifier],
                    quote_type = "Price",
                    # Case sensitive: the field value must match that of the equivalent recipe field exactly
                    field = "mid",
                ),
#                effective_at = to_date(f"{price['Date']}")
 #               effective_at = to_date(price["Date"],"MM/dd/yyyy")
               effective_at = '2023-01-17'
            ),
            metric_value = models.MetricValue(value = price["Price"], unit = price["Currency"]),
            # Bonds require a scale factor equivalent to par
            scale_factor = par
        )
        for index, price in dataframe.iterrows()
    }
    
    # Upsert quotes into LUSID
    upsert_quotes_response = quotes_api.upsert_quotes(scope = orders_scope, request_body = quotes)

    if upsert_quotes_response.failed == {}:
        print(f"{len(upsert_quotes_response.values)} {provider} prices successfully loaded into LUSID.")

    else:
        print(f"Some {provider} failures occurred. {len(upsert_quotes_response.failed)} did not get loaded into LUSID.")
        
# Load equity prices from EDI using FIGI identifiers
load_quotes(equity_prices_df, "Edi", "Figi", 1)


{'e51510fc-b629-43a6-9d12-0c3d325f6a88': {'lineage': None,
  'metric_value': {'unit': 'USD', 'value': 96.0},
  'quote_id': {'effective_at': '2023-01-17',
               'quote_series_id': {'field': 'mid',
                                   'instrument_id': 'BBG000BVPXP1',
                                   'instrument_id_type': 'Figi',
                                   'price_source': None,
                                   'provider': 'Edi',
                                   'quote_type': 'Price'}},
  'scale_factor': 1},
 'ce12b38f-d007-4c27-b7f3-3eae68a992a9': {'lineage': None,
  'metric_value': {'unit': 'USD', 'value': 97.0},
  'quote_id': {'effective_at': '2023-01-17',
               'quote_series_id': {'field': 'mid',
                                   'instrument_id': 'BBG000BVPXP1',
                                   'instrument_id_type': 'Figi',
                                   'price_source': None,
                                   'provider': 'Edi',
                    

Some Edi failures occurred. 2 did not get loaded into LUSID.


In [14]:
help(to_date)

Help on function to_date in module lusidtools.lpt.lpt:

to_date(date, **kwargs)
    # Utilities to convert YYYY-MM-DD strings to and from UTC dates.



### Upsert Quotes
Load some sample data for our instrument. We're uploading a quote for today, as market orders are converted to holdings by assuming that they're filled at the instant we make the query and take the latest market price for it.

In [15]:
quotes_df = pd.read_excel("data/quotes.xlsx")
quotes_df['date']=date.today()
quotes_df.head()

,date,ticker,name,figi,Sector,open_price,close_price,client_internal
0,2023-02-15,AMZN,Amazon Nasdaq,BBG000BVPXP1,Equity Investment Instruments,147.00,149.75,imd_34634534
1,2023-02-15,NaN,Anglian Water 40LV,NaN,Fixed Income,110.93,112.20,imd_13579246
2,2023-02-15,ZIZ21,ICE Silver 5000oz Dec21,NaN,Commodities,24.30,25.40,imd_12457801


In [16]:
quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$ClientInternal",
    "quote_id.effective_at": "date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "client_internal",
    "metric_value.unit": "$USD",
}

In [17]:
quotes_mapping["quote_id.quote_series_id.field"] ="$mid"
quotes_mapping["metric_value.value"] = "close_price"

result = load_from_data_frame(
    api_factory = api_factory,
    scope=orders_scope,
    data_frame=quotes_df,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes"
)
succ, failed, errors = format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

,success,failed,errors
0,3,0,0


## Check the initial portfolio holdings
We see just a single, material holding in the instrument.

In [18]:
# DD: 1/20 modified w/ addition of final 2 params
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings_with_orders(scope=orders_scope,code=orders_portfolio,property_keys=["Instrument/default/Name"],recipe_id_scope="holdings_with_live_orders",recipe_id_code="test_recipe")

response_df=lusid_response_to_data_frame(executed_holdings, rename_properties=True)
response_df[["instrument_uid", "holding_type", "units", "cost.amount", "cost.currency"]].style.format({"units":"{:20,.0f}","cost.amount": "{:20,.2f}"})

,instrument_uid,holding_type,units,cost.amount,cost.currency
0,LUID_00003D5T,P,"5,000","825,000.00",USD
1,LUID_00003D4X,O,500,"56,100.00",USD
2,CCY_USD,OC,"-56,100","-56,100.00",USD
3,LUID_00003D5T,O,"-1,000","-100,500.00",USD
4,CCY_USD,OC,"100,500","100,500.00",USD
5,LUID_00003D5T,O,"-3,000","-449,250.00",USD
6,CCY_USD,OC,"449,250","449,250.00",USD
7,LUID_00003D5T,O,223,"33,394.25",USD
8,CCY_USD,OC,"-33,394","-33,394.25",USD
9,LUID_00003D4W,O,"5,000","127,000.00",USD


## Post orders into LUSID

In this section, we post some [orders](https://support.finbourne.com/how-does-lusid-support-the-trade-lifecycle) into LUSID. We have a number of orders for Amazon stock to demonstrate the processing of market, limit, and stop orders. In addtion we have an order for a corporate bond.

In [19]:
orders_df = pd.read_csv('data/live_orders.csv')
orders_df

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,order_id,side,type,state,limit_price,limit_currency
0,us_long,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,1000,100.0,USD,ORD001,sell,limit,new,100.50,USD
1,us_long,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,1000,95.0,USD,ORD002,buy,limit,new,96.25,USD
2,us_long,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,3000,NaN,USD,ORD003,sell,market,new,NaN,NaN
3,us_long,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,1000,NaN,USD,ORD004,buy,market,new,NaN,NaN
4,us_long,Anglian_Water_40LV,imd_13579246,XS0089553282,NaN,500,NaN,USD,ORD005,buy,market,new,NaN,NaN
5,us_long,ICE_Silver_5000oz_Dec21,imd_12457801,NaN,NaN,5000,NaN,USD,ORD006,buy,market,new,NaN,NaN


In [20]:
order_requests = defaultdict(list)
order_sets = defaultdict(list)
responses = []

for index, order in orders_df.iterrows():
    
    request = models.OrderRequest(
            id=models.ResourceId(
                scope=orders_scope,
                code=order['order_id']
            ),
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/ClientInternal': order['client_internal']
            },
            properties={},
            portfolio_id=models.ResourceId(
                scope=orders_scope,
                code=orders_portfolio
            ),
            state=order['state'],
            type=order['type'],
            price=models.CurrencyAndAmount(
                        amount=0 if pd.isna(order['price']) else order['price'],
                        currency=order['currency']),

        #  limit price definition
            limit_price=models.CurrencyAndAmount(
                        amount=order['limit_price'],
                        currency=order['limit_currency']
            )
            if not pd.isna(order['limit_price']) and not pd.isna(order['limit_currency']) else None
        
        #  stop order definition
#            stop_price=models.CurrencyAndAmount(
#                        amount=order['stop_price'],
#                        currency=order['stop_currency']
#            )
#            if not pd.isna(order['stop_price']) and not pd.isna(order['stop_currency']) else None
    )
    
    request=models.OrderSetRequest(
        order_requests=[request]           
    )

    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        order_set_request=request
    )
    
    responses.append(response.values[0])

# display(responses)
attributes=[(o.id.code,o.instrument_identifiers['Instrument/default/ClientInternal'],
             o.lusid_instrument_id,o.side,o.type,o.state,o.quantity,o.price.amount,o.price.currency,
             o.limit_price.amount if o.limit_price is not None else "N/A",
             o.limit_price.currency if o.limit_price is not None else "N/A") for o in responses]
pd.DataFrame(attributes, columns=['order_id','client_internal','lusid_instrument_id','side','type',
                                  'state','quantity','price','currency','lim px',
                                  'lim ccy']).style.format({"quantity":"{:20,.0f}","price": "{:20,.2f}"})

,order_id,client_internal,lusid_instrument_id,side,type,state,quantity,price,currency,lim px,lim ccy
0,ORD001,imd_34634534,LUID_00003D5T,sell,limit,new,"1,000",100.00,USD,100.500000,USD
1,ORD002,imd_34634534,LUID_00003D5T,buy,limit,new,"1,000",95.00,USD,96.250000,USD
2,ORD003,imd_34634534,LUID_00003D5T,sell,market,new,"3,000",0.00,USD,N/A,N/A
3,ORD004,imd_34634534,LUID_00003D5T,buy,market,new,"1,000",0.00,USD,N/A,N/A
4,ORD005,imd_13579246,LUID_00003D4X,buy,market,new,500,0.00,USD,N/A,N/A
5,ORD006,imd_12457801,LUID_00003D4W,buy,market,new,"5,000",0.00,USD,N/A,N/A


### Post allocations into LUSID
In this section, we post some allocations into LUSID. These allocations represent full or partial allocations against a subset of the originating orders.

In [21]:
allocations_df = pd.read_csv('data/amazon_allocations.csv')
allocations_df

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,allocation_id,originating_order,state,side,type
0,us_long,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,777,961,USD,ALLOC001-for-ORD004,ORD004,partial allocation,buy,market


In [22]:
allocation_requests = defaultdict(list)
allocation_sets = defaultdict(list)
responses = []

for index, allocation in allocations_df.iterrows():
    
    portfolio = allocation['portfolio']

    request = models.AllocationRequest(
            id=models.ResourceId(
                scope=orders_scope,
                code=allocation['allocation_id']
            ),
            allocated_order_id = models.ResourceId(
                scope=orders_scope,
                code=allocation['originating_order']
            ),
            quantity=allocation['quantity'],
            side=allocation['side'],
            instrument_identifiers={
                'Instrument/default/ClientInternal': allocation['client_internal']
            },
            properties={},
            portfolio_id=models.ResourceId(
                scope=orders_scope,
                code=orders_portfolio
            ),
            state=allocation['state'],
            type=allocation['type'],
            price=models.CurrencyAndAmount(
                        amount=allocation['price'],
                        currency=allocation['currency']))
    
    request=models.AllocationSetRequest(
        allocation_requests=[request]           
    )

    response = api_factory.build(lusid.api.AllocationsApi).upsert_allocations(
        allocation_set_request=request
    )
    
    responses.append(response.values[0])

attributes=[(o.id.code,o.instrument_identifiers['Instrument/default/ClientInternal'],o.lusid_instrument_id,o.side,o.type,o.state,o.quantity,o.price.amount) for o in responses]
pd.DataFrame(attributes, columns=['allocation_id','client_internal','lusid_instrument_id','side','type','state','quantity','price']).style.format({"quantity":"{:20,.0f}","price": "${:20,.2f}"})

,allocation_id,client_internal,lusid_instrument_id,side,type,state,quantity,price
0,ALLOC001-for-ORD004,imd_34634534,LUID_00003D5T,buy,market,partial allocation,777,$ 961.00


## Check the portfolio holdings with orders
Finally we can ask LUSID to give us a live holdings view that includes contributions from outstanding orders of market or limit type.

Note that we've added an order of "stop" type (ORD003), which is currently unsupported. We won't be able to generate holdings for it, but will see a warning returned alongside the successful results.

We'll see Order (O) and OrderContra (OC) holdings for the security and cash commitments of each outstanding order. Partially- and fully-allocated orders will also have Allocation (L) and AllocationContra (LC) holdings to represent allocations against them, and have their quantities suitably reduced.

In [23]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings_with_orders(
    scope=orders_scope, 
    code=orders_portfolio, 
    property_keys=["Instrument/default/Name"], 
    recipe_id_scope=recipe_scope, 
    recipe_id_code=recipe_code)

response_df = lusid_response_to_data_frame(executed_holdings, rename_properties=True)

response_df.rename(columns = {'transaction.transaction_price.price':'price','Name(default-Properties)':'instrument','cost.amount':'cost','cost.currency':'ccy','cost_portfolio_ccy.amount':'pfolio_cost'}, inplace = True)
response_df['transaction.transaction_id'] = response_df['transaction.transaction_id'].fillna('')
grouped=response_df.groupby('transaction.transaction_id')

for name,group in grouped:
    display(group[["instrument", "holding_type", "units", "cost", "ccy", "pfolio_cost",  "price"]].style.format({"price":"{:20,.2f}","units":"{:20,.0f}","cost": "{:20,.2f}","pfolio_cost": "{:20,.2f}"}).set_caption(name))


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
0,Amazon_Nasdaq_AMZN,P,"5,000","825,000.00",USD,"825,000.00",nan


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
13,Amazon_Nasdaq_AMZN,L,777,"746,697.00",USD,"746,697.00",961.00
14,USD,LC,"-746,697","-746,697.00",USD,"-746,697.00",961.00


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
5,Amazon_Nasdaq_AMZN,O,"-1,000","-100,500.00",USD,"-100,500.00",100.50
6,USD,OC,"100,500","100,500.00",USD,"100,500.00",100.50


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
9,Amazon_Nasdaq_AMZN,O,"1,000","96,250.00",USD,"96,250.00",96.25
10,USD,OC,"-96,250","-96,250.00",USD,"-96,250.00",96.25


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
3,Amazon_Nasdaq_AMZN,O,"-3,000","-449,250.00",USD,"-449,250.00",149.75
4,USD,OC,"449,250","449,250.00",USD,"449,250.00",149.75


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
1,Amazon_Nasdaq_AMZN,O,223,"33,394.25",USD,"33,394.25",149.75
2,USD,OC,"-33,394","-33,394.25",USD,"-33,394.25",149.75


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
7,Anglian_Water_40LV,O,500,"56,100.00",USD,"56,100.00",112.20
8,USD,OC,"-56,100","-56,100.00",USD,"-56,100.00",112.20


,instrument,holding_type,units,cost,ccy,pfolio_cost,price
11,ICE_Silver_5000oz_Dec21,O,"5,000","127,000.00",USD,"127,000.00",25.40
12,USD,OC,"-127,000","-127,000.00",USD,"-127,000.00",25.40


We should see warnings for:
- ORD003, an order of an unsupported order type (stop - only market and limit are supported)

In [24]:
display(executed_holdings.warnings)

[]